# 🚀 Day 5: Agent2Agent Communication & Production Deployment

**Welcome to Day 5 of the Kaggle 5-day Agents course!**

This comprehensive notebook combines two critical aspects of production AI agents:

## Part 1: Agent2Agent (A2A) Communication
Learn how to build **multi-agent systems** where different agents communicate and collaborate using the **Agent2Agent (A2A) Protocol**.

## Part 2: Production Deployment
Deploy your agents to production using **Vertex AI Agent Engine** and learn about production best practices.

---

## 🎯 What You'll Learn

### Part 1: A2A Communication
- ✅ Understand the A2A protocol and when to use it vs sub-agents
- ✅ Learn common A2A architecture patterns (cross-framework, cross-language, cross-organization)
- ✅ Expose an ADK agent via A2A using `to_a2a()`
- ✅ Consume remote agents using `RemoteA2aAgent`
- ✅ Build a product catalog integration system

### Part 2: Production Deployment
- ✅ Build a production-ready ADK agent
- ✅ Deploy your agent to Vertex AI Agent Engine using the ADK CLI
- ✅ Test your deployed agent with Python SDK
- ✅ Monitor and manage deployed agents in Google Cloud Console
- ✅ Understand Memory Bank for long-term agent memory
- ✅ Learn cost management and cleanup best practices


---

# 📚 INFORMATION: Understanding A2A Protocol

## 🤔 The Problem

As you build more complex AI systems, you'll find that:
- **A single agent can't do everything** - Specialized agents for different domains work better
- **You need agents to collaborate** - Customer support needs product data, order systems need inventory info
- **Different teams build different agents** - You want to integrate agents from external vendors
- **Agents may use different languages/frameworks** - You need a standard communication protocol

## ✅ The Solution: A2A Protocol

The [Agent2Agent (A2A) Protocol](https://a2a-protocol.org/) is a **standard** that allows agents to:
- ✨ **Communicate over networks** - Agents can be on different machines
- ✨ **Use each other's capabilities** - One agent can call another agent like a tool
- ✨ **Work across frameworks** - Language/framework agnostic
- ✨ **Maintain formal contracts** - Agent cards describe capabilities

## 🗺️ Common A2A Architecture Patterns

The A2A protocol is particularly useful in three scenarios:

![When to choose A2A?](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_01.png)

1. **Cross-Framework Integration**: ADK agent communicating with other agent frameworks
2. **Cross-Language Communication**: Python agent calling Java or Node.js agent  
3. **Cross-Organization Boundaries**: Your internal agent integrating with external vendor services

## 💡 A2A vs Local Sub-Agents: Decision Table

| Factor | Use A2A | Use Local Sub-Agents |
|--------|---------|---------------------|
| **Agent Location** | External service, different codebase | Same codebase, internal |
| **Ownership** | Different team/organization | Your team |
| **Network** | Agents on different machines | Same process/machine |
| **Performance** | Network latency acceptable | Need low latency |
| **Language/Framework** | Cross-language/framework needed | Same language |
| **Contract** | Formal API contract required | Internal interface |
| **Example** | External vendor product catalog | Internal order processing steps |

## 📋 What This Tutorial Demonstrates

We'll build a practical e-commerce integration:
1. **Product Catalog Agent** (exposed via A2A) - External vendor service that provides product information
2. **Customer Support Agent** (consumer) - Your internal agent that helps customers by querying product data

```text
┌──────────────────────┐           ┌──────────────────────┐
│ Customer Support     │  ─A2A──▶  │ Product Catalog      │
│ Agent (Consumer)     │           │ Agent (Vendor)       │
│ Your Company         │           │ External Service     │
│ (localhost:8000)     │           │ (localhost:8001)     │
└──────────────────────┘           └──────────────────────┘
```

## 🏷️ Understanding Vertex AI Agent Engine

Vertex AI Agent Engine is Google Cloud's fully managed service for deploying AI agents:

### Key Features:
- **Fully managed** service specifically for AI agents
- **Auto-scaling** with session management built-in
- **Easy deployment** using ADK CLI
- **Free tier available** for getting started

### Why Deploy Agents?

Your agent only lives in your notebook and development environment. When you stop your notebook session, it stops working. Your teammates can't access it. Your users can't interact with it. **Deployment solves this!**

### Deployment Architecture

```text
┌─────────────────┐     Deploy      ┌──────────────────────┐
│  Your Agent     │  ─────────▶     │  Vertex AI           │
│  (Development)  │                  │  Agent Engine        │
│  - agent.py     │                  │  - Auto-scaling      │
│  - tools        │                  │  - Session mgmt      │
│  - config       │                  │  - Production ready  │
└─────────────────┘                  └──────────────────────┘
```

---

# ⚙️ Setup

Before we begin, let's set up the environment for both tasks.

## Install Dependencies

The Kaggle Notebooks environment includes pre-installed ADK libraries. For your own environment, you would run:

```python
pip install -q google-adk[a2a]
```

## Configure API Keys and Credentials

### For Task 1 (A2A): Gemini API Key

**1. Get your API key**

Create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

1. In the top menu bar, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key and click "Save".
4. Ensure the checkbox is selected.

### For Task 2 (Deployment): Google Cloud Credentials

**1. Create Google Cloud Account**

- [Sign up here](https://cloud.google.com/free) - New users get $300 in free credits
- Watch this [3-minute setup video](https://youtu.be/-nUAQq_evxc)

**2. Link Google Cloud Account**

1. In the top menu bar, select `Add-ons` then `Google Cloud SDK`.
2. Click on `Link Account`
3. Select your Google Cloud Account
4. Attach to the notebook

In [ ]:
import os
import json
import random
import requests
import subprocess
import time
import uuid
import vertexai

from kaggle_secrets import UserSecretsClient
from google.adk.agents import Agent, LlmAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent, AGENT_CARD_WELL_KNOWN_PATH
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from vertexai import agent_engines

import warnings
warnings.filterwarnings("ignore")

# Configure Gemini API Key for Task 1
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API Key configured")
except Exception as e:
    print(f"🔒 Please add 'GOOGLE_API_KEY' to Kaggle secrets. Details: {e}")

# Configure Google Cloud credentials for Task 2
try:
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)
    print("✅ Google Cloud credentials configured")
except Exception as e:
    print(f"⚠️ Google Cloud credentials not configured (needed for Task 2): {e}")

print("\n✅ All imports and setup completed successfully!")

## Configure Retry Options

When working with LLMs, retry options help handle transient errors automatically.

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("✅ Retry configuration set")

---

# 🤝 TASK 1: Agent2Agent (A2A) Communication

In this task, you'll build a multi-agent system where agents communicate using the A2A protocol.

## What We'll Build

![A2A Architecture](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_02.png)

**Flow:**
1. Customer asks Support Agent about a product
2. Support Agent calls Product Catalog Agent via A2A
3. Product Catalog Agent returns data
4. Support Agent responds to customer

## Tutorial Steps

1. Create the Product Catalog Agent
2. Expose it via A2A
3. Start the server
4. Create the Customer Support Agent (consumer)
5. Test the communication
6. Understand what happened

## Step 1.1: Create the Product Catalog Agent

We'll create a **Product Catalog Agent** that provides product information. This simulates an external vendor's service.

In [ ]:
def get_product_info(product_name: str) -> str:
    """Get product information for a given product.

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Product information as a string
    """
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": 'Dell XPS 15, $1,299, In Stock (45 units), 15.6" display, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": 'iPad Air, $599, In Stock (28 units), 10.9" display, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34" Monitor, $499, Out of Stock, Expected: Next week',
    }

    product_lower = product_name.lower().strip()

    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"


product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Be professional and helpful.
    """,
    tools=[get_product_info],
)

print("✅ Product Catalog Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_product_info()")
print("   Ready to be exposed via A2A...")

## Step 1.2: Expose the Agent via A2A

Now we'll use `to_a2a()` to make the Product Catalog Agent accessible via the A2A protocol.

### What `to_a2a()` does:
- 🔧 Wraps your agent in an A2A-compatible server
- 📋 Auto-generates an **agent card** (capabilities contract)
- 🌐 Serves the agent card at `/.well-known/agent-card.json`
- ✨ Handles all A2A protocol details

In [ ]:
product_catalog_a2a_app = to_a2a(
    product_catalog_agent, 
    port=8001
)

print("✅ Product Catalog Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8001")
print("   Agent card will be at: http://localhost:8001/.well-known/agent-card.json")
print("   Ready to start the server...")

## Step 1.3: Start the Product Catalog Agent Server

We'll start the server in the background so it can accept requests.

In [ ]:
# Save the product catalog agent to a file
product_catalog_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

def get_product_info(product_name: str) -> str:
    """Get product information for a given product."""
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": "Dell XPS 15, $1,299, In Stock (45 units), 15.6\\" display, 16GB RAM, 512GB SSD",
        "macbook pro 14": "MacBook Pro 14\\", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD",
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": "iPad Air, $599, In Stock (28 units), 10.9\\" display, 64GB",
        "lg ultrawide 34": "LG UltraWide 34\\" Monitor, $499, Out of Stock, Expected: Next week",
    }
    
    product_lower = product_name.lower().strip()
    
    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"

product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Be professional and helpful.
    """,
    tools=[get_product_info]
)

app = to_a2a(product_catalog_agent, port=8001)
'''

with open("/tmp/product_catalog_server.py", "w") as f:
    f.write(product_catalog_agent_code)

print("📝 Product Catalog agent code saved to /tmp/product_catalog_server.py")

# Start server in background
server_process = subprocess.Popen(
    ["uvicorn", "product_catalog_server:app", "--host", "localhost", "--port", "8001"],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)

print("🚀 Starting Product Catalog Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get("http://localhost:8001/.well-known/agent-card.json", timeout=1)
        if response.status_code == 200:
            print(f"\n✅ Product Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️ Server may not be ready yet. Check manually if needed.")

globals()["product_catalog_server_process"] = server_process

## Step 1.4: View the Auto-Generated Agent Card

Let's examine the agent card that was automatically generated.

In [ ]:
try:
    response = requests.get("http://localhost:8001/.well-known/agent-card.json", timeout=5)

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Product Catalog Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")

## Step 1.5: Create the Customer Support Agent (Consumer)

Now we'll create a Customer Support Agent that **consumes** the Product Catalog Agent using `RemoteA2aAgent`.

In [ ]:
# Create a RemoteA2aAgent - this is a client-side proxy
remote_product_catalog_agent = RemoteA2aAgent(
    name="product_catalog_agent",
    description="Remote product catalog agent from external vendor that provides product information.",
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Product Catalog Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

In [ ]:
# Create the Customer Support Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="customer_support_agent",
    description="A customer support assistant that helps customers with product inquiries and information.",
    instruction="""
    You are a friendly and professional customer support agent.
    
    When customers ask about products:
    1. Use the product_catalog_agent sub-agent to look up product information
    2. Provide clear answers about pricing, availability, and specifications
    3. If a product is out of stock, mention the expected availability
    4. Be helpful and professional!
    
    Always get product information from the product_catalog_agent before answering customer questions.
    """,
    sub_agents=[remote_product_catalog_agent],
)

print("✅ Customer Support Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Product Catalog Agent via A2A)")
print("   Ready to help customers!")

## Step 1.6: Test A2A Communication

Let's test the agent-to-agent communication!

In [ ]:
async def test_a2a_communication(user_query: str):
    """Test A2A communication between agents."""
    session_service = InMemorySessionService()
    app_name = "support_app"
    user_id = "demo_user"
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    runner = Runner(
        agent=customer_support_agent, 
        app_name=app_name, 
        session_service=session_service
    )

    test_content = types.Content(parts=[types.Part(text=user_query)])

    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 Support Agent response:")
    print("-" * 60)

    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)


# Run the test
print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about the iPhone 15 Pro? Is it in stock?")

## Step 1.7: Understanding What Just Happened

### A2A Communication Flow

When you ran the tests above, here's the detailed step-by-step flow of how the agents communicated:

![](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_03.png)

**A2A Protocol Communication:**

Behind the scenes, here's what happens at the protocol level:
- **RemoteA2aAgent** sends HTTP POST requests to the `/tasks` endpoint on `http://localhost:8001`
- Request and response data follow the [A2A Protocol Specification](https://a2a-protocol.org/latest/specification/)
- Data is exchanged in standardized JSON format
- The protocol ensures any A2A-compatible agent (regardless of language/framework) can communicate

This standardization is what makes cross-organization, cross-language agent communication possible!

---

**What happened:**
1. **Customer** asked about the iPhone 15 Pro
2. **Customer Support Agent** (LlmAgent) received the question and decided it needs product information
3. **Support Agent** delegated to the `product_catalog_agent` sub-agent
4. **RemoteA2aAgent** (client-side proxy) translated this into an A2A protocol request
5. The A2A request was sent over HTTP to `http://localhost:8001` (highlighted in yellow)
6. **Product Catalog Agent** (server) received the request and called `get_product_info("iPhone 15 Pro")`
7. **Product Catalog Agent** returned the product information via A2A response
8. **RemoteA2aAgent** received the response and passed it back to the Support Agent
9. **Support Agent** formulated a final answer with the product details
10. **Customer** received the complete, helpful response

### Key Benefits Demonstrated

1. **Transparency**: Support Agent doesn't "know" Product Catalog Agent is remote
2. **Standard Protocol**: Uses A2A standard - any A2A-compatible agent works
3. **Easy Integration**: Just one line: `sub_agents=[remote_product_catalog_agent]`
4. **Separation of Concerns**: Product data lives in Catalog Agent (vendor), support logic in Support Agent (your company)

### Real-World Applications

This pattern enables:
- **Microservices**: Each agent is an independent service
- **Third-party Integration**: Consume agents from external vendors (e.g., product catalogs, payment processors)
- **Cross-language**: Product Catalog Agent could be Java, Support Agent Python
- **Specialized Teams**: Vendor maintains catalog, your team maintains support agent
- **Cross-Organization**: Vendor hosts catalog on their infrastructure, you integrate via A2A